# Get started with Metrics Tracking

This notebook demonstrates how to use MLFlow to:
- Log metrics, params, and artifacts to MLFlow.
- Log, register, and load models using a local MLflow Tracking Server.
- Interact with the MLflow Tracking Server using the MLflow fluent API.
- Perform inference on Pandas DataFrames by loading models as generic Python Functions (pyfunc).

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv
    
import joblib
import mlflow
from mlflow.models import infer_signature
import mlflow.sklearn
import pandas as pd
from pathlib import Path
from sklearn import ensemble, model_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [5]:
%%bash 

echo $MLFLOW_TRACKING_URI

https://tracking.mlflow-e00pn4p3f6ns0rynwa.backbone-e00msp-e2e.msp.eu-north1.nebius.cloud


# Train model and calculate metrics

## Load Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [21]:
# Download original dataset with: python src/load_data.py 

raw_data = pd.read_csv("../data/raw_data.csv")
raw_data.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Prepare data

In [22]:
target = 'cnt'
prediction = 'prediction'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]

In [23]:
sample_data = raw_data.set_index('dteday').loc['2011-01-01 00:00:00':'2011-01-28 23:00:00'].reset_index()

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    sample_data[numerical_features + categorical_features],
    sample_data[target],
    test_size=0.3
)

print(X_train.shape)
print(X_test.shape)

(415, 10)
(179, 10)


## Log Dataset with `log_inputs` API 

In [24]:
from mlflow.data.pandas_dataset import PandasDataset

In [29]:
# Create an instance of a PandasDataset
dataset = mlflow.data.from_pandas(
    raw_data, name="bike-sharing-dataset", targets="cnt"
)

# Log the Dataset to an MLflow run by using the `log_input` API
with mlflow.start_run() as run:
    mlflow.log_input(dataset, context="training")

# Retrieve the run information
logged_run = mlflow.get_run(run.info.run_id)

# Retrieve the Dataset object
logged_dataset = logged_run.inputs.dataset_inputs[0].dataset

# View some of the recorded Dataset information
print(f"Dataset name: {logged_dataset.name}")
print(f"Dataset digest: {logged_dataset.digest}")
print(f"Dataset profile: {logged_dataset.profile}")
print(f"Dataset schema: {logged_dataset.schema}")

/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/10/17 21:17:12 INFO mlflow.tracking._tracking_service.client: 🏃 View run exultant-gnu-577 at: http://localhost:5001/#/experiments/0/runs/4b7a3e6454974a48872a3b8904f23421.
2024/10/17 21:17:12 INFO mlflow.tracki

Dataset name: bike-sharing-dataset
Dataset digest: 1d75aeb9
Dataset profile: {"num_rows": 17379, "num_elements": 295443}
Dataset schema: {"mlflow_colspec": [{"type": "long", "name": "instant", "required": true}, {"type": "string", "name": "dteday", "required": true}, {"type": "long", "name": "season", "required": true}, {"type": "long", "name": "yr", "required": true}, {"type": "long", "name": "mnth", "required": true}, {"type": "long", "name": "hr", "required": true}, {"type": "long", "name": "holiday", "required": true}, {"type": "long", "name": "weekday", "required": true}, {"type": "long", "name": "workingday", "required": true}, {"type": "long", "name": "weathersit", "required": true}, {"type": "double", "name": "temp", "required": true}, {"type": "double", "name": "atemp", "required": true}, {"type": "double", "name": "hum", "required": true}, {"type": "double", "name": "windspeed", "required": true}, {"type": "long", "name": "casual", "required": true}, {"type": "long", "name": 

In [31]:
logged_dataset

<Dataset: digest='1d75aeb9', name='bike-sharing-dataset', profile='{"num_rows": 17379, "num_elements": 295443}', schema=('{"mlflow_colspec": [{"type": "long", "name": "instant", "required": true}, '
 '{"type": "string", "name": "dteday", "required": true}, {"type": "long", '
 '"name": "season", "required": true}, {"type": "long", "name": "yr", '
 '"required": true}, {"type": "long", "name": "mnth", "required": true}, '
 '{"type": "long", "name": "hr", "required": true}, {"type": "long", "name": '
 '"holiday", "required": true}, {"type": "long", "name": "weekday", '
 '"required": true}, {"type": "long", "name": "workingday", "required": true}, '
 '{"type": "long", "name": "weathersit", "required": true}, {"type": "double", '
 '"name": "temp", "required": true}, {"type": "double", "name": "atemp", '
 '"required": true}, {"type": "double", "name": "hum", "required": true}, '
 '{"type": "double", "name": "windspeed", "required": true}, {"type": "long", '
 '"name": "casual", "required": tru

In [30]:
 # Loading the dataset's source
dataset_source = mlflow.data.get_source(logged_dataset)

local_dataset = dataset_source.load()

print(f"The local file where the data has been downloaded to: {local_dataset}")

# Load the data again
loaded_data = pd.read_csv(local_dataset, delimiter=";")

NotImplementedError: 

### Create dataset from URL 

In [32]:
import mlflow
import pandas as pd
from mlflow.data.pandas_dataset import PandasDataset


dataset_source_url = "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv"
raw_data = pd.read_csv(dataset_source_url, delimiter=";")

# Create an instance of a PandasDataset
dataset = mlflow.data.from_pandas(
    raw_data, source=dataset_source_url, name="wine quality - white", targets="quality"
)

# Log the Dataset to an MLflow run by using the `log_input` API
with mlflow.start_run() as run:
    mlflow.log_input(dataset, context="training")

# Retrieve the run information
logged_run = mlflow.get_run(run.info.run_id)

# Retrieve the Dataset object
logged_dataset = logged_run.inputs.dataset_inputs[0].dataset

# View some of the recorded Dataset information
print(f"Dataset name: {logged_dataset.name}")
print(f"Dataset digest: {logged_dataset.digest}")
print(f"Dataset profile: {logged_dataset.profile}")
print(f"Dataset schema: {logged_dataset.schema}")


/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/mlflow/data/dataset_source_registry.py:149: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv'. Exception: 
  return _dataset_source_registry.resolve(
/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling I

Dataset name: wine quality - white
Dataset digest: 2a1e42c4
Dataset profile: {"num_rows": 4898, "num_elements": 58776}
Dataset schema: {"mlflow_colspec": [{"type": "double", "name": "fixed acidity", "required": true}, {"type": "double", "name": "volatile acidity", "required": true}, {"type": "double", "name": "citric acid", "required": true}, {"type": "double", "name": "residual sugar", "required": true}, {"type": "double", "name": "chlorides", "required": true}, {"type": "double", "name": "free sulfur dioxide", "required": true}, {"type": "double", "name": "total sulfur dioxide", "required": true}, {"type": "double", "name": "density", "required": true}, {"type": "double", "name": "pH", "required": true}, {"type": "double", "name": "sulphates", "required": true}, {"type": "double", "name": "alcohol", "required": true}, {"type": "long", "name": "quality", "required": true}]}


In [37]:
logged_dataset

<Dataset: digest='2a1e42c4', name='wine quality - white', profile='{"num_rows": 4898, "num_elements": 58776}', schema=('{"mlflow_colspec": [{"type": "double", "name": "fixed acidity", "required": '
 'true}, {"type": "double", "name": "volatile acidity", "required": true}, '
 '{"type": "double", "name": "citric acid", "required": true}, {"type": '
 '"double", "name": "residual sugar", "required": true}, {"type": "double", '
 '"name": "chlorides", "required": true}, {"type": "double", "name": "free '
 'sulfur dioxide", "required": true}, {"type": "double", "name": "total sulfur '
 'dioxide", "required": true}, {"type": "double", "name": "density", '
 '"required": true}, {"type": "double", "name": "pH", "required": true}, '
 '{"type": "double", "name": "sulphates", "required": true}, {"type": '
 '"double", "name": "alcohol", "required": true}, {"type": "long", "name": '
 '"quality", "required": true}]}'), source=('{"url": '
 '"https://raw.githubusercontent.com/mlflow/mlflow/master/tests/d

In [38]:
 # Loading the dataset's source
dataset_source = mlflow.data.get_source(logged_dataset)

local_dataset = dataset_source.load()

print(f"The local file where the data has been downloaded to: {local_dataset}")

# Load the data again
loaded_data = pd.read_csv(local_dataset, delimiter=";")
loaded_data.head(3)

The local file where the data has been downloaded to: /var/folders/28/ly259mjn413_8g33c4n20tvc0000gn/T/tmpbsd1bfro/winequality-white.csv


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6


## Train a  Model

In [40]:
model = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)
model.fit(X_train, y_train) 

model_path = Path('../models/model.joblib')
joblib.dump(model, model_path)

['../models/model.joblib']

In [41]:
model

RandomForestRegressor(n_estimators=50, random_state=0)

## Calculate Metrics

In [42]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

preds = model.predict(X_test)

me = mean_squared_error(y_test, preds)
mae = mean_absolute_error(y_test, preds)

print(me, mae)

610.6395955307263 16.00659217877095


# Autologging

## Set up MLFlow

In [45]:
MLFLOW_TRACKING_URI = "http://localhost:5001"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

## Autologging 

- https://mlflow.org/docs/latest/tracking/autolog.html

In [47]:
mlflow.autolog()

with mlflow.start_run():

    model = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)
    model.fit(X_train, y_train) 

2024/10/18 12:11:23 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/10/18 12:11:23 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: Failed to initialize NVML, skip logging GPU metrics: NVML Shared Library Not Found.
2024/10/18 12:11:23 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2024/10/18 12:11:23 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training 

In [49]:
# Get MLflow Run instance from autologged results


autolog_run = mlflow.last_active_run()
print(autolog_run)

<Run: data=<RunData: metrics={'training_mean_absolute_error': 4.8026506024096385,
 'training_mean_squared_error': 59.29453975903615,
 'training_r2_score': 0.972273516570996,
 'training_root_mean_squared_error': 7.700294783905104,
 'training_score': 0.972273516570996}, params={'bootstrap': 'True',
 'ccp_alpha': '0.0',
 'criterion': 'squared_error',
 'max_depth': 'None',
 'max_features': '1.0',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'monotonic_cst': 'None',
 'n_estimators': '50',
 'n_jobs': 'None',
 'oob_score': 'False',
 'random_state': '0',
 'verbose': '0',
 'warm_start': 'False'}, tags={'estimator_class': 'sklearn.ensemble._forest.RandomForestRegressor',
 'estimator_name': 'RandomForestRegressor',
 'mlflow.log-model.history': '[{"run_id": "bedc09d6cc32499cacbfdefef5e5dcaf", '
                             '"artifact_path": "model", "utc_time_created": '

# Hyperparameters Tuning

- https://mlflow.org/docs/latest/getting-started/quickstart-2/index.html

In [54]:
!pip install keras hyperopt tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.2/236.2 MB 21.2 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 21.7 MB/s eta 0:00:0031m22.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 19.8 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.0
    Uninstalling ml_dtypes-0.5.0:
      Successfully uninstalled ml_dtypes-0.5.0


In [55]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

2024/10/18 17:09:23 WARNING mlflow.tracking.fluent: Exception raised while enabling autologging for tensorflow: cannot import name 'cast' from partially initialized module 'keras.src.backend' (most likely due to a circular import) (/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/keras/src/backend/__init__.py)
2024/10/18 17:09:24 WARNING mlflow.utils.autologging_utils: MLflow keras autologging is known to be compatible with 3.0.2 <= keras <= 3.5.0, but the installed version is 3.6.0. If you encounter errors during autologging, try upgrading / downgrading keras to a compatible version, or try upgrading MLflow.
2024/10/18 17:09:24 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.


In [56]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)


In [57]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}


In [61]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [62]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


2024/10/18 17:12:48 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: Failed to initialize NVML, skip logging GPU metrics: NVML Shared Library Not Found.
2024/10/18 17:12:48 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


  0%|                                                                                                                                         | 0/8 [00:00<?, ?trial/s, best loss=?]

2024/10/18 17:12:49 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: Failed to initialize NVML, skip logging GPU metrics: NVML Shared Library Not Found.

2024/10/18 17:12:49 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.



Epoch 1/3                                                                                                                                                                           

 1/46 ━━━━━━━━━━━━━━━━━━━━ 7s 177ms/step - loss: 37.2229 - root_mean_squared_error: 6.1011                                                                 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 34.4049 - root_mean_squared_error: 5.8650 - val_loss: 32.7360 - val_root_mean_squared_error: 5.7215

Epoch 2/3                                                                                                                                                                           

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 31.6046 - root_mean_squared_error: 5.6218                                                                  
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 664us/step - loss: 32.0812 - root_mean_squared_error: 5.6640 - val_loss: 31.0345 - val_root_mean_squared_error: 5.5709

Epoch 3/3                 

2024/10/18 17:12:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run indecisive-seal-857 at: http://localhost:5001/#/experiments/937421358963430873/runs/297d53cea6114989be252f10cb3e1b9c.

2024/10/18 17:12:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/937421358963430873.

2024/10/18 17:12:55 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...

2024/10/18 17:12:55 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!



...

Epoch 1/3                                                                                                                                                                           

 1/46 ━━━━━━━━━━━━━━━━━━━━ 8s 182ms/step - loss: 32.0271 - root_mean_squared_error: 5.6593                                                                 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 28.7957 - root_mean_squared_error: 5.3647 - val_loss: 23.3741 - val_root_mean_squared_error: 4.8347

Epoch 2/3                                                                                                                                                                           

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 23.1192 - root_mean_squared_error: 4.8082                                                                  
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 664us/step - loss: 21.6813 - root_mean_squared_error: 4.6550 - val_loss: 17.4359 - val_root_mean_squared_error: 4.1756

Epoch 3/3                 

2024/10/18 17:13:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run rare-cub-950 at: http://localhost:5001/#/experiments/937421358963430873/runs/635ee47eb42b43f4a8977058819a1f65.

2024/10/18 17:13:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/937421358963430873.

2024/10/18 17:13:41 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...

2024/10/18 17:13:41 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:52<00:00,  6.51s/trial, best loss: 0.7342399954795837]


2024/10/18 17:13:43 INFO mlflow.tracking._tracking_service.client: 🏃 View run ambitious-rook-238 at: http://localhost:5001/#/experiments/937421358963430873/runs/09614d4736b14cffbd1933807e621deb.
2024/10/18 17:13:43 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/937421358963430873.
2024/10/18 17:13:43 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/10/18 17:13:44 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


Best parameters: {'lr': 0.04617237922487885, 'momentum': 0.2370473230583292}
Best eval rmse: 0.7342399954795837


# Deployment and Serving the model

- https://mlflow.org/docs/latest/getting-started/quickstart-2/index.html#serve-the-model-locally
- https://mlflow.org/docs/latest/deployment/index.html
- https://mlflow.org/docs/latest/deployment/deploy-model-to-kubernetes/index.html 

Deployment options (prod)
- Flask
- MLServer - scalability and reliability of Kubernetes to serve your model at scale

Deployment plugins https://mlflow.org/docs/latest/plugins.html#deployment-plugins 
- mlflow-torchserve
- mlflow-algorithmia

In [69]:
# mlflow models serve -m "models:/wine-quality/1" --port 5002

In [1]:
%%bash 

curl -d '{"dataframe_split": {
"columns": ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"],
"data": [[7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8]]}}' \
-H 'Content-Type: application/json' -X POST localhost:5002/invocations


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (7) Failed to connect to localhost port 5002 after 0 ms: Couldn't connect to server


CalledProcessError: Command 'b'\ncurl -d \'{"dataframe_split": {\n"columns": ["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"],\n"data": [[7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8]]}}\' \\\n-H \'Content-Type: application/json\' -X POST localhost:5002/invocations\n'' returned non-zero exit status 7.

In [3]:
from mlflow.deployments import get_deploy_client

client = get_deploy_client("http://127.0.0.1:5000")

data = {
    "prompt": (
        "What would happen if an asteroid the size of "
        "a basketball encountered the Earth traveling at 0.5c? "
        "Please provide your answer in .rst format for the purposes of documentation."
    ),
    "temperature": 0.5,
    "max_tokens": 1000,
    "n": 1,
    "frequency_penalty": 0.2,
    "presence_penalty": 0.2,
}

client.predict(endpoint="completions", inputs=data)

{'id': 'chatcmpl-ALt6WHiak4YAqYcJp55p0TvWscgNN',
 'object': 'text_completion',
 'created': 1729780072,
 'model': 'gpt-4o-mini-2024-07-18',
 'choices': [{'index': 0,
   'text': '```rst\nAsteroid Impact Scenario: Basketball-Sized Asteroid at 0.5c\n==========================================================\n\nOverview\n--------\nThis document outlines the hypothetical scenario of a basketball-sized asteroid (approximately 24 cm in diameter) traveling at 0.5 times the speed of light (0.5c) encountering Earth. \n\nKey Variables\n-------------\n- **Asteroid Size**: Diameter of approximately 24 cm\n- **Speed**: 0.5c (approximately 149,896,229 m/s)\n- **Mass**: The mass of the asteroid can be estimated based on its volume and density. Assuming an average density similar to that of rock (around 2,500 kg/m³), the mass would be approximately:\n  \n  .. math::\n     \\text{Volume} = \\frac{4}{3} \\pi r^3 \\\\\n     r = \\frac{24 \\text{ cm}}{2} = 12 \\text{ cm} = 0.12 \\text{ m} \\\\\n     \\text{

In [4]:
for endpoint in client.list_endpoints():
    print(endpoint)

name='ai-studio-chat' endpoint_type='llm/v1/chat' model=RouteModelInfo(name='meta-llama/Meta-Llama-3.1-8B-Instruct', provider='openai') endpoint_url='http://127.0.0.1:5000/gateway/ai-studio-chat/invocations' limit=None
name='completions' endpoint_type='llm/v1/completions' model=RouteModelInfo(name='gpt-4o-mini', provider='openai') endpoint_url='http://127.0.0.1:5000/gateway/completions/invocations' limit=Limit(calls=10, key=None, renewal_period='minute')
name='chat' endpoint_type='llm/v1/chat' model=RouteModelInfo(name='gpt-4o-mini', provider='openai') endpoint_url='http://127.0.0.1:5000/gateway/chat/invocations' limit=None
name='embeddings' endpoint_type='llm/v1/embeddings' model=RouteModelInfo(name='text-embedding-ada-002', provider='openai') endpoint_url='http://127.0.0.1:5000/gateway/embeddings/invocations' limit=None


/Users/mikhailrozhkov/.pyenv/versions/3.11.6/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [6]:
response = client.predict(
    endpoint="ai-studio-chat",
    inputs={"messages": [{"role": "user", "content": "Tell me a joke about rabbits"}]},
)
print(response)


{'id': 'chat-6e5254c91efd41f79638450419d3a6df', 'object': 'chat.completion', 'created': 1729780315, 'model': 'meta-llama/Meta-Llama-3.1-8B-Instruct', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Here's one:\n\nWhy did the rabbit go to the doctor?\n\nBecause he had hare-loss! (get it?)", 'tool_calls': []}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 16, 'completion_tokens': 24, 'total_tokens': 40}}


In [7]:
response = client.predict(
    endpoint="completions",
    inputs={"prompt": "Tell me a joke about rabbits"},
)
print(response)

{'id': 'chatcmpl-ALuC02z8aKlL9W1x0ymOdHro4br6K', 'object': 'text_completion', 'created': 1729784256, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'text': 'What do you call a line of rabbits hopping backward? \n\nA receding hare-line! 🐰', 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 13, 'completion_tokens': 21, 'total_tokens': 34}}


In [8]:
response

{'id': 'chatcmpl-ALuC02z8aKlL9W1x0ymOdHro4br6K',
 'object': 'text_completion',
 'created': 1729784256,
 'model': 'gpt-4o-mini-2024-07-18',
 'choices': [{'index': 0,
   'text': 'What do you call a line of rabbits hopping backward? \n\nA receding hare-line! 🐰',
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 13, 'completion_tokens': 21, 'total_tokens': 34}}

# Nebius AI Studio <> MLflow integration

In [71]:
!pip install 'mlflow[genai]' tf-keras textstat

In [74]:
import os
import json
from typing import List, Dict, Any
import mlflow
from openai import OpenAI
from mlflow.models import EvaluationResult


In [75]:
%load_ext dotenv
%dotenv

In [ ]:
class NebiusAIPredictor:
    def __init__(self, model_name: str = "meta-llama/Meta-Llama-3.1-70B-Instruct"):
        self.client = OpenAI(
            base_url="https://api.studio.nebius.ai/v1/",
            api_key=os.environ.get("NEBIUS_API_KEY"),
        )
        self.model_name = model_name

    def predict(self, data: List[str]) -> List[str]:
        """
        Generate predictions using Nebius AI Studio model
        """
        results = []
        for prompt in data:
            try:
                completion = self.client.chat.completions.create(
                    model=self.model_name,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.6
                )
                response = completion.choices[0].message.content
                results.append(response)
            except Exception as e:
                print(f"Error processing prompt: {e}")
                results.append("")
        return results


MODEL = "meta-llama/Meta-Llama-3.1-8B-Instruct"
TEMPERATURE=0.0
TOP_P=0.95
MAX_TOKENS=2048

# Initialize predictor
predictor = NebiusAIPredictor(MODEL)

# mlflow.pyfunc

In [76]:
import mlflow


# define a custom model
class MyModel(mlflow.pyfunc.PythonModel):
    def predict(self, context, model_input, params=None):
        return self.my_custom_function(model_input, params)

    def my_custom_function(self, model_input, params=None):
        # do something with the model input
        return 0


some_input = 1
# save the model
with mlflow.start_run():
    model_info = mlflow.pyfunc.log_model(artifact_path="model", python_model=MyModel())

# load the model
loaded_model = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)
print(type(loaded_model))  # <class 'mlflow.pyfunc.model.PyFuncModel'>
unwrapped_model = loaded_model.unwrap_python_model()
print(type(unwrapped_model))  # <class '__main__.MyModel'>

# does not work, only predict() is exposed
# print(loaded_model.my_custom_function(some_input))
print(unwrapped_model.my_custom_function(some_input))  # works
print(loaded_model.predict(some_input))  # works

# works, but None is needed for context arg
print(unwrapped_model.predict(None, some_input))


2024/10/24 12:18:25 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: Failed to initialize NVML, skip logging GPU metrics: NVML Shared Library Not Found.
2024/10/24 12:18:25 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2024/10/24 12:18:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/24 12:18:26 INFO mlflow.tracking._tracking_service.client: 🏃 View run fearless-ox-129 at: http://localhost:5001/#/experiments/937421358963430873/runs/ba7377aa040b4ba6bcf12a45692b4197.
2024/10/24 12:18:26 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5001/#/experiments/937421358963430873.
2024/10/24 12:18:26 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/10/24 12:18:26 

<class 'mlflow.pyfunc.PyFuncModel'>
<class '__main__.MyModel'>
0
0
0


In [77]:
print(loaded_model.my_custom_function(some_input))

AttributeError: 'PyFuncModel' object has no attribute 'my_custom_function'

In [78]:
loaded_model.predict

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.pyfunc.model
  run_id: ba7377aa040b4ba6bcf12a45692b4197

## Prompt Engineering 

- https://mlflow.org/docs/latest/llms/prompt-engineering/index.html 